In [1]:
import math
import numpy as np
from scipy import linalg

EPS = 1.E-32

import sys
sys.path.append("/usr/local/")
import cytnx
from cytnx import cytnx_extension as cyx


def physical_dimension_to_spin(d):
    return str((d - 1) // 2) if (d - 1) % 2 == 0 else str(d - 1) + '/2'


def spin_to_physical_dimension(spin):
    s = int(spin.split('/')[0]) / int(spin.split('/')[1]) if ('/' in spin) else int(spin)
    return int(2 * s + 1)

In [2]:
def getSpinOperators(spin):
    """Returns tuple of 3 spin operators and a unit matrix for given value of spin."""
    s = int(spin.split('/')[0]) / int(spin.split('/')[1]) if ('/' in spin) else int(spin)
    d = int(2 * s + 1)
    eye = cytnx.from_numpy(np.eye(d, dtype=complex))
    sx = cytnx.zeros([d,d], dtype=cytnx.Type.ComplexDouble)
    sy = cytnx.zeros([d,d], dtype=cytnx.Type.ComplexDouble)
    sz = cytnx.zeros([d,d], dtype=cytnx.Type.ComplexDouble)
    for a in range(d):
        if a != 0:
            sx[a, a - 1] = ((s + 1) * (2 * a) - (a + 1) * a)**0.5 / 2
            sy[a, a - 1] = 1j * ((s + 1) * (2 * a) - (a + 1) * a)**0.5 / 2
        if a != d - 1:
            sx[a, a + 1] = ((s + 1) * (2 * a + 2) - (a + 2) * (a + 1))**0.5 / 2
            sy[a, a + 1] = -1j * ((s + 1) * (2 * a + 2) - (a + 2) * (a + 1))**0.5 / 2
        sz[a, a] = s - a
    if spin == '1/2':
        sx *= 2
        sy *= 2
        sz *= 2

    return sx, sy, sz, eye

In [3]:
def create_loop_gas_operator(spin):
    """Returns loop gas (LG) operator Q_LG for spin=1/2 or spin=1 Kitaev model."""

    tau_tensor = np.zeros((2, 2, 2), dtype=complex)  # tau_tensor_{i j k}

    if '/' in spin:
        tau_tensor[0][0][0] = - 1j
    else:
        tau_tensor[0][0][0] = 1

    tau_tensor[0][1][1] = tau_tensor[1][0][1] = tau_tensor[1][1][0] = 1

    sx, sy, sz, one = get_spin_operators(spin)
    d = one.shape[0]

    Q_LG = np.zeros((d, d, 2, 2, 2), dtype=complex)  # Q_LG_{s s' i j k}

    u_gamma = None
    if spin == "1/2":
        u_gamma = (sx, sy, sz)
    elif '/' in spin:
        u_gamma = tuple(map(lambda x: -1j * linalg.expm(1j * math.pi * x), (sx, sy, sz)))
    else:
        u_gamma = tuple(map(lambda x: linalg.expm(1j * math.pi * x), (sx, sy, sz)))

    for i in range(2):
        for j in range(2):
            for k in range(2):
                temp = np.eye(d)
                if i == 0:
                    temp = temp @ u_gamma[0]
                if j == 0:
                    temp = temp @ u_gamma[1]
                if k == 0:
                    temp = temp @ u_gamma[2]
                for s in range(d):
                    for sp in range(d):
                        Q_LG[s][sp][i][j][k] = tau_tensor[i][j][k] * temp[s][sp]

    return Q_LG

In [4]:
def createLoopGasOperator(spin):
    """Returns loop gas (LG) operator Q_LG for spin=1/2 or spin=1 Kitaev model."""

    tau_tensor = cytnx.zeros((2, 2, 2), dtype=cytnx.Type.ComplexDouble)  # tau_tensor_{i j k}

    if '/' in spin:
        tau_tensor[0,0,0] = - 1j
    else:
        tau_tensor[0,0,0] = 1

    tau_tensor[0,1,1] = tau_tensor[1,0,1] = tau_tensor[1,1,0] = 1

    sx, sy, sz, one = getSpinOperators(spin)
    d = one.shape()[0]

    Q_LG = cytnx.zeros((d, d, 2, 2, 2), dtype=cytnx.Type.ComplexDouble)  # Q_LG_{s s' i j k}

    u_gamma = None
    if spin == "1/2":
        u_gamma = (sx, sy, sz)
    elif '/' in spin:
        u_gamma = tuple(map(lambda x: -1j * cytnx.linalg.Exp(1j * math.pi * x), (sx, sy, sz)))
    else:
        u_gamma = tuple(map(lambda x: cytnx.linalg.Exp(1j * math.pi * x), (sx, sy, sz)))

    for i in range(2):
        for j in range(2):
            for k in range(2):
                temp = cytnx.from_numpy(np.eye(d))
                print(temp[i,j].item())
                if i == 0:
                    temp = cytnx.linalg.Matmul(temp, u_gamma[0])
                if j == 0:
                    temp = cytnx.linalg.Matmul(temp, u_gamma[1])
                if k == 0:
                    temp = cytnx.linalg.Matmul(temp, u_gamma[2])
    
                for s in range(d):
                    for sp in range(d):
                        print(temp[s,sp].item(), tau_tensor[i,j,k].item())
                        Q_LG[s,sp,i,j,k] = tau_tensor[i,j,k] * temp[s,sp]
                        test = cyx.CyTensor(Q_LG,2)
                        test = cyx.Contract(test, test)
#                         print(test.item())
#     print('tau_tensor = ', tau_tensor)
#     print('temp = ', temp)
    return Q_LG, tau_tensor, 

# Error

### This will give wrong answer (temp goes to infinity)

In [5]:
sx, sy, sz, one = getSpinOperators('1')
u_gamma = tuple(map(lambda x: cytnx.linalg.ExpH(1j * math.pi * x), (sx, sy, sz)))
print('u_gamma = ', u_gamma)
temp = cytnx.from_numpy(np.eye(3))
print('temp = ', temp)
temp = cytnx.linalg.Matmul(temp, u_gamma[0])
print('temp = ', temp)

temp = cytnx.linalg.Matmul(temp, u_gamma[1])
# temp = cytnx.linalg.Matmul(temp, u_gamma[2])
print('temp = ', temp)



u_gamma =  
Total elem: 9
type  : Complex Double (Complex Float64)
cytnx device: CPU
Shape : (3,3)
[[6.29598e+00+0.00000e+00j 0.00000e+00-8.16619e+00j -5.29598e+00+0.00000e+00j ]
 [0.00000e+00-8.16619e+00j -1.15920e+01+0.00000e+00j 0.00000e+00+8.16619e+00j ]
 [-5.29598e+00+0.00000e+00j 0.00000e+00+8.16619e+00j 6.29598e+00+0.00000e+00j ]]



Total elem: 9
type  : Complex Double (Complex Float64)
cytnx device: CPU
Shape : (3,3)
[[6.29598e+00+0.00000e+00j -8.16619e+00+0.00000e+00j 5.29598e+00+0.00000e+00j ]
 [-8.16619e+00+0.00000e+00j 1.15920e+01+0.00000e+00j -8.16619e+00+0.00000e+00j ]
 [5.29598e+00+0.00000e+00j -8.16619e+00+0.00000e+00j 6.29598e+00+0.00000e+00j ]]



Total elem: 9
type  : Complex Double (Complex Float64)
cytnx device: CPU
Shape : (3,3)
[[1.00000e+00+0.00000e+00j 0.00000e+00+0.00000e+00j 0.00000e+00+0.00000e+00j ]
 [0.00000e+00+0.00000e+00j 1.00000e+00+0.00000e+00j 0.00000e+00+0.00000e+00j ]
 [0.00000e+00+0.00000e+00j 0.00000e+00+0.00000e+00j 1.00000e+00+0.00000e+00j ]]


In [6]:
sx, sy, sz, one = getSpinOperators('1')
# print(sx, sy,sz) It is accurate 
ux = cytnx.linalg.ExpH(math.pi*sx )

Sx, Sy, Sz, One = sx.numpy(), sy.numpy(), sz.numpy(), one.numpy()

testNp = linalg.expm(1j*math.pi*Sy)
testCt = cytnx.linalg.ExpH(1j*math.pi*sy )

_ = 1j*math.pi
testCt2 = cytnx.linalg.ExpH( _*sy )
# print(cytnx.linalg.Matmul(testCt2, testCt2))
print(testNp, testCt, testCt2)

[[ 1.66260249e-16+0.j  8.54725382e-17+0.j  1.00000000e+00+0.j]
 [ 1.66260249e-16+0.j -1.00000000e+00-0.j  2.07825311e-16+0.j]
 [ 1.00000000e+00+0.j -2.47047960e-16+0.j  3.08809950e-17+0.j]] 
Total elem: 9
type  : Complex Double (Complex Float64)
cytnx device: CPU
Shape : (3,3)
[[6.29598e+00+0.00000e+00j -8.16619e+00+0.00000e+00j 5.29598e+00+0.00000e+00j ]
 [-8.16619e+00+0.00000e+00j 1.15920e+01+0.00000e+00j -8.16619e+00+0.00000e+00j ]
 [5.29598e+00+0.00000e+00j -8.16619e+00+0.00000e+00j 6.29598e+00+0.00000e+00j ]]


 
Total elem: 9
type  : Complex Double (Complex Float64)
cytnx device: CPU
Shape : (3,3)
[[6.29598e+00+0.00000e+00j -8.16619e+00+0.00000e+00j 5.29598e+00+0.00000e+00j ]
 [-8.16619e+00+0.00000e+00j 1.15920e+01+0.00000e+00j -8.16619e+00+0.00000e+00j ]
 [5.29598e+00+0.00000e+00j -8.16619e+00+0.00000e+00j 6.29598e+00+0.00000e+00j ]]



